In [31]:
# Import the main libraries
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
from geemap import ee_to_pandas


Unzipping geemap-master.zip ...
Data downloaded to: C:\Users\naveenr\Downloads\geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


In [32]:
# Authenticate and initialize the Earth Engine API
# This may require you to sign in to your Google account the first time
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

    
print('Libraries imported and GEE initialized successfully!')

Libraries imported and GEE initialized successfully!


In [33]:
### Define the Region of Interest (ROI)

In [34]:
# Define the coordinates for the Krishna River Delta
aoi_geometry = ee.Geometry.Point(80.85, 16.35)

# Create a square bounding box around the point (80km x 80km)
aoi = aoi_geometry.buffer(40000).bounds()

# Create an interactive map centered on our AOI
Map = geemap.Map(center=[16.35, 80.85], zoom=9)
Map.addLayer(aoi, {'color': 'red', 'fillColor': '00000000'}, 'Area of Interest: Krishna Delta')
Map

Map(center=[16.35, 80.85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

# Filter and Extract Sentinel-1 collection

In [35]:
# Filter the Sentinel-1 GRD Image Collection for the monsoon season
s1_collection = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(aoi)
    .filterDate('2024-05-01', '2024-11-30')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
)

# Print the number of images found
image_count = s1_collection.size().getInfo()
print(f'Found {image_count} images in the collection.')

Found 46 images in the collection.


In [36]:
# Create a Median Composite

In [37]:
# Create a median composite from the collection
s1_composite = s1_collection.median().clip(aoi)

# Define visualization parameters for single-band SAR data
vis_params = {'min': -25, 'max': 0}

# Add the VV and VH bands to the map to compare them
Map.addLayer(s1_composite.select('VV'), vis_params, 'VV Polarization')
Map.addLayer(s1_composite.select('VH'), vis_params, 'VH Polarization')
Map

Map(center=[16.35, 80.85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [38]:
 ### Apply a Speckle Filter

In [39]:
# Apply a focal median filter to reduce speckle
s1_filtered = s1_composite.focal_median(radius=50, units='meters')

# Create a split-panel map to compare the original vs. filtered image
left_layer = geemap.ee_tile_layer(s1_composite.select('VV'), vis_params, 'Original')
right_layer = geemap.ee_tile_layer(s1_filtered.select('VV'), vis_params, 'Speckle Filtered')

split_map = geemap.Map(center=[16.35, 80.85], zoom=10)
split_map.split_map(left_layer, right_layer)
split_map

Map(center=[16.35, 80.85], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [40]:
# Create an RGB Color Composite

In [41]:
# Create the VV/VH ratio band
vv_vh_ratio = s1_filtered.select('VV').divide(s1_filtered.select('VH')).rename('VV_VH_Ratio')

# Add the ratio band to our filtered image
s1_rgb = s1_filtered.addBands(vv_vh_ratio)

# Define RGB visualization parameters
rgb_vis_params = {'min': [-20, -25, 1], 'max': [0, -10, 15], 'bands': ['VV', 'VH', 'VV_VH_Ratio']}

# Add the final RGB composite to the map
Map.addLayer(s1_rgb, rgb_vis_params, 'RGB Composite')
Map

Map(center=[16.35, 80.85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [42]:
## Generate a Time-Series Chart for a Rice Paddy

In [50]:
# Define a point inside a known rice-growing area
rice_paddy_point = ee.Geometry.Point(80.7, 16.4)

# Get the time-series data for this point from the original collection
ts_data = s1_collection.getRegion(rice_paddy_point, scale=10).getInfo()
print(ts_data)
# Convert the data to a Pandas DataFrame
df = geemap.ee_to_pandas(ts_data)
df['datetime'] = pd.to_datetime(df['time'], unit='ms')

# Plot the time-series for VH polarization
plt.figure(figsize=(12, 6))
plt.plot(df['datetime'], df['VH'], marker='o', linestyle='-', color='green')
plt.title('VH Backscatter Time-Series for a Rice Paddy in Krishna Delta')
plt.xlabel('Date')
plt.ylabel('VH Backscatter (dB)')
plt.grid(True)
plt.show()

[['id', 'longitude', 'latitude', 'time', 'VV', 'VH', 'angle'], ['S1A_IW_GRDH_1SDV_20240516T003109_20240516T003134_053889_068CBA_2EA8', 80.70001880158459, 16.399958237235428, 1715819469000, -14.093380891643605, -21.1766422578248, 35.6246223449707], ['S1A_IW_GRDH_1SDV_20240528T003109_20240528T003134_054064_0692D2_784E', 80.70001880158459, 16.399958237235428, 1716856269000, -11.807660736538903, -18.650654468659585, 35.611873626708984], ['S1A_IW_GRDH_1SDV_20240609T003109_20240609T003134_054239_0698E0_32A2', 80.70001880158459, 16.399958237235428, 1717893069000, -15.75120165512914, -22.639941849620584, 35.61698532104492], ['S1A_IW_GRDH_1SDV_20240621T003108_20240621T003133_054414_069EEC_2567', 80.70001880158459, 16.399958237235428, 1718929868000, -12.330360858295482, -15.378496197473064, 35.611663818359375], ['S1A_IW_GRDH_1SDV_20240703T003107_20240703T003132_054589_06A503_EEB2', 80.70001880158459, 16.399958237235428, 1719966667000, -10.336843008273721, -14.973016657697894, 35.62510299682617],

AttributeError: module 'geemap' has no attribute 'ee_to_pandas'

In [ ]:
!conda install -c conda-forge geemap

